In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip -q /content/gdrive/MyDrive/Amazonai/train_split.zip -d .

replace ./train_split.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!unzip -q /content/gdrive/MyDrive/Amazonai/valid_split.zip -d .

replace ./valid_split.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!pip install fasttext

In [ ]:
import fasttext

In [ ]:
# Training the fastText classifier
#model = fasttext.train_supervised('train_full.txt', wordNgrams = 2,loss = 'hs',dim = 200, epoch = 3)
model = fasttext.train_supervised(input='train_split.txt', autotuneValidationFile='valid_split.txt', autotuneDuration=9000)

#model.test('test.txt')                      

# Predicting on a single input
#model.predict(ds.iloc[2, 0])

# Save the trained model
#model.save_model('model.bin')

In [ ]:

# Evaluating performance
get_model_stats(model)

In [ ]:
model.save_model('model_autotune.bin')

In [ ]:
!cp model_autotune.bin /content/gdrive/MyDrive/Amazonai

In [ ]:
!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
!unzip dataset52a7b21.zip
!rm dataset52a7b21.zip

--2021-08-01 11:27:27--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.128.202
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.128.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1061576029 (1012M) [binary/octet-stream]
Saving to: ‘dataset52a7b21.zip’

dataset52a7b21.zip  100%[===================>]   1012M  52.3MB/s    in 18s     

2021-08-01 11:27:45 (54.8 MB/s) - ‘dataset52a7b21.zip’ saved [1061576029/1061576029]

Archive:  dataset52a7b21.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/.~lock.train.csv#  


In [ ]:
import pandas as pd
import csv
import numpy as np

In [ ]:
df_test = pd.read_csv("dataset/test.csv", escapechar="\\", quoting=csv.QUOTE_NONE)
#mask = df_test.index[df_test.TITLE.isnull() & df_test.DESCRIPTION.isnull() & df_test.BULLET_POINTS.isnull()]
#df_test.drop(mask, 0, inplace=True)
df_test.fillna("", inplace=True)
df_test

,PRODUCT_ID,TITLE,DESCRIPTION,BULLET_POINTS,BRAND
0,1,"Command 3M Small Kitchen Hooks, White, Decorat...",Sale Unit: PACK,[INCLUDES - 9 hooks and 12 small indoor strips...,Command
1,2,O'Neal Jump Hardware JAG Unisex-Adult Glove (B...,Synthetic leather palm with double-layer thumb...,[Silicone printing for a better grip. Long las...,O'Neal
2,3,"NFL Detroit Lions Portable Party Fridge, 15.8 ...",Boelter Brands lets you celebrate your favorit...,[Runs on 12 Volt DC Power or 110 Volt AC Power...,Boelter Brands
3,4,Panasonic Single Line KX-TS880MX Corded Phone ...,Features: 50 Station Phonebook Corded Phone Al...,Panasonic Landline Phones doesn't come with a ...,Panasonic
4,5,Zero Baby Girl's 100% Cotton Innerwear Bloomer...,"Zero Baby Girl Panties Set. 100% Cotton, Breat...","[Zero Baby Girl Panties, Pack of 6, 100% Cotto...",Zero
...,...,...,...,...,...
110770,110771,AAHNA E MALL OneBlade Hybrid Trimmer Shaver An...,<p>1-All In One Hyper Advanced Smart Rechargea...,"[Unique One Blade can style, trim and shave, w...",Generic
110771,110772,Grin Health N99 Anti Pollution Reusable Washab...,"<p>SIZE GUIDE : M - (35- 65 Kg), L - (49- 72 K...",[PROTECTION: Filtration rate up to ≥99 percent...,Grin Health
110772,110773,Asian Army Pink Ultra reusable respirator clot...,Asian HyperProtect A95 masks have been enginee...,[Reusable and environment friendly: These mask...,ASIAN
110773,110774,IM Safe 3 Ply Non-Woven Disposable Surgical Fa...,This 3 Ply Disposable face mask is manufacture...,[3 Ply Mask: Genuine 3 Ply Mask. 25 GSM Spun B...,Intermarket


In [ ]:
test_list = df_test.values.tolist()

In [ ]:
# !pip3 install autocorrect
!python -m nltk.downloader punkt
!python -m nltk.downloader stopwords
!python -m nltk.downloader wordnet

#from autocorrect import Speller
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 

lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
#spell = Speller(lang='en')

count = 0 

def text_cleaner(text):
    rules = [
        {r'>\s+': u'>'},  # remove spaces after a tag opens or closes
        {r'\s+': u' '},  # replace consecutive spaces
        {r'\s*<br\s*/?>\s*': u''},  # newline after a <br>
        {r'</(div)\s*>\s*': u''},  # newline after </p> and </div> and <h1/>...
        {r'</(p|h\d)\s*>\s*': u''},  # newline after </p> and </div> and <h1/>...
        {r'<head>.*<\s*(/head|body)[^>]*>': u''},  # remove <head> to </head>
        {r'<a\s+href="([^"]+)"[^>]*>.*</a>': r'\1'},  # show links instead of texts
        {r'[ \t]*<[^<]*?/?>': u''},  # remove remaining tags
        {r'^\s+': u''}  # remove spaces at the beginning
    ]
    for rule in rules:
      for (k, v) in rule.items():
          regex = re.compile(k)
          text = regex.sub(v, text)
    text = text.rstrip()
    return text.lower()


def removeNoise(text):
  global count
  count = count + 1
  if count%10000==0:
    print(count)
  text = text_cleaner(text)
  tokens = word_tokenize(text.lower())
  tokens = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in tokens] 
  tokens = list(filter(None, tokens))
  # print(tokens)
  # spelling_tokens = [spell(w) for w in tokens]
  filtered_tokens = [w for w in tokens if not w in stop_words]
  filtered_tokens = [lemmatizer.lemmatize(w) for w in filtered_tokens]
  return filtered_tokens

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]  

In [ ]:
X_test = [' '.join(row[1:]) for row in test_list]
print(len(X_test))

product_id = [row[0] for row in test_list]

110775


In [ ]:
X_test = [' '.join(removeNoise(x)) for x in X_test]

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000


In [41]:
!cp /content/gdrive/MyDrive/Amazonai/model_split1.bin .

In [42]:
model_split1 = fasttext.load_model("model_split1.bin")

In [43]:
Y_test = model_split1.predict(X_test)

In [44]:
predicted = [int(i[0][9:]) for i in Y_test[0]]
print(predicted)

[1140, 2321, 11, 125, 4645, 33, 2321, 4, 4, 15772, 2321, 4, 1045, 2321, 14478, 2321, 7848, 787, 962, 2900, 155, 1079, 155, 1045, 2321, 4, 604, 1045, 6565, 2321, 1045, 2321, 106, 3050, 3175, 3175, 1045, 3175, 2321, 993, 11, 890, 1045, 1045, 1045, 4708, 796, 796, 1045, 4306, 4, 29622, 1045, 6696, 81, 322, 10422, 2229, 739, 1045, 4, 1319, 1287, 1295, 1045, 1045, 5616, 2673, 840, 1045, 82, 361, 507, 17397, 805, 289, 340, 439, 322, 4, 1045, 1084, 1209, 465, 1287, 1052, 977, 1239, 1045, 796, 1667, 1045, 1239, 5932, 1045, 1045, 289, 1045, 840, 1658, 1671, 1045, 1045, 1045, 800, 1045, 506, 8082, 4, 396, 1024, 1045, 1045, 1045, 1045, 1295, 3867, 2229, 1045, 1045, 4980, 1045, 419, 2029, 1295, 1045, 5278, 818, 6, 1045, 1045, 1045, 8078, 1240, 155, 2229, 748, 328, 2273, 11655, 2229, 5398, 1045, 1295, 787, 1295, 1045, 6544, 818, 22948, 507, 787, 36, 6, 1239, 8594, 324, 5061, 507, 507, 2385, 2650, 1922, 16174, 1281, 6, 289, 977, 1239, 3175, 796, 5338, 782, 2318, 9643, 858, 2181, 834, 834, 834, 1052,

In [45]:
result = pd.DataFrame(
    {
        'PRODUCT_ID': product_id,
        'BROWSE_NODE_ID': predicted   
    }
)

In [46]:
result

,PRODUCT_ID,BROWSE_NODE_ID
0,1,1140
1,2,2321
2,3,11
3,4,125
4,5,4645
...,...,...
110770,110771,4368
110771,110772,13568
110772,110773,800
110773,110774,800


In [47]:
result.to_csv('result_split1.csv',index=False)

In [48]:
from google.colab import files
files.download('result_split1.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>